In [118]:
import requests
import json
import numpy as np
from hashlib import pbkdf2_hmac
import base64
import binascii
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import plotly.express as px
import re
import string
import pandas as pd

In [119]:
def getCryptoInitializers():
    metadata = requests.get('https://maps.amtrak.com/rttl/js/RoutesList.json').text
    public_key_index = 0
    for route in json.loads(metadata):
        try:
            public_key_index += route['ZoomLevel']
        except KeyError:
            pass
    
    public_key_metadata = requests.get('https://maps.amtrak.com/rttl/js/RoutesList.v.json').text
    public_key_metadata = json.loads(public_key_metadata)
    public_key = public_key_metadata['arr'][public_key_index]
    
    salt = public_key_metadata['s'][len(public_key_metadata['s'][0])]
    init_vec = public_key_metadata['v'][len(public_key_metadata['v'][0])]

    return {
        'PUBLIC_KEY': public_key,
        'CRYPTO_SALT': binascii.unhexlify(salt),
        'CRYPTO_IV': binascii.unhexlify(init_vec)
    }

In [120]:
cryptoInitializers = getCryptoInitializers()
MASTER_SEGMENT = 88

In [121]:
def decrypt(data, keyDerivationPassword):
    cipherText = base64.b64decode(data)

    key = pbkdf2_hmac('sha1', keyDerivationPassword.encode(), cryptoInitializers['CRYPTO_SALT'], 1000, 16)

    decipher = AES.new(key=key, mode=AES.MODE_CBC, iv=cryptoInitializers['CRYPTO_IV'])

    return decipher.decrypt(cipherText).decode()

In [122]:
rawData = requests.get('https://maps.amtrak.com/services/MapDataService/trains/getTrainsData').text

privateKeyCipher = rawData[-MASTER_SEGMENT:]
encryptedTrainData = rawData[:-MASTER_SEGMENT]

In [123]:
private_key = decrypt(privateKeyCipher, cryptoInitializers['PUBLIC_KEY']).split('|')[0]

In [124]:
train_data_string = decrypt(encryptedTrainData, private_key)
train_data = json.loads(''.join([c for c in train_data_string if c in string.printable]))

In [125]:
def get_station_indicies():
    keys = train_data['features'][0]['properties'].keys()
    look_from_idx = len('Station')
    indices = [int(key[look_from_idx:]) for key in keys if 'Station' in key]
    indices.sort()
    return indices

In [126]:
def does_train_stop_at_station(train_dict, station_code):
    station_indices = get_station_indicies()
    for idx in station_indices:
        try:
            station = json.loads(train_dict['Station' + str(idx)])
        except TypeError:
            continue
        if station['code'] == station_code:
            return True
        
def get_station_info(train_dict, station_code):
    station_indices = get_station_indicies()
    for idx in station_indices:
        try:
            station = json.loads(train_dict['Station' + str(idx)])
        except TypeError:
            continue
        if station['code'] == station_code:
            return station
        
def find_denver_trains():
    trains = [train['properties'] for train in train_data['features']]
    return [train for train in trains if does_train_stop_at_station(train, 'DEN')]

In [127]:
denver_trains = find_denver_trains()

In [128]:
def parse_station_data(station_data):
    return_dict = {}
    if 'postdep' in station_data.keys():
        return_dict['status'] = 'departed'
        return_dict['actual_arrival'] = station_data['postarr']
        return_dict['scheduled_arrival'] = station_data['scharr']
        return_dict['estimated_arrival'] = None
        return_dict['actual_departure'] = station_data['postdep']
        return_dict['scheduled_departure'] = station_data['schdep']
        return_dict['estimated_departure'] = None
    elif 'postarr' in station_data.keys():
        return_dict['status'] = 'arrived'
        return_dict['actual_arrival'] = station_data['postarr']
        return_dict['scheduled_arrival'] = station_data['scharr']
        return_dict['estimated_arrival'] = None
        return_dict['actual_departure'] = None
        return_dict['scheduled_departure'] = station_data['schdep']
        return_dict['estimated_departure'] = station_data['estdep']
    else:
        return_dict['status'] = 'enroute'
        return_dict['actual_arrival'] = None
        return_dict['scheduled_arrival'] = station_data['scharr']
        return_dict['estimated_arrival'] = station_data['estarr']
        return_dict['actual_departure'] = None
        return_dict['scheduled_departure'] = station_data['schdep']
        return_dict['estimated_departure'] = station_data['estdep']
    return return_dict

In [129]:
def get_train_summary(train_dict):
    denver_station_summary = parse_station_data(get_station_info(train_dict, 'DEN'))
    return_dict = {
        'train_num': train_dict['TrainNum'],
        'heading': train_dict['Heading'],
        'dest': train_dict['DestCode'],
        'origin': train_dict['OrigCode'],
        'route': train_dict['RouteName'],
        'state': train_dict['TrainState'],
        'velocity': train_dict['Velocity'],
    }
    return_dict.update(denver_station_summary)
    return return_dict

In [130]:
train_df = pd.DataFrame.from_records([get_train_summary(train_dict) for train_dict in denver_trains])
for col in ['actual_arrival', 'scheduled_arrival', 'estimated_arrival', 'actual_departure', 'scheduled_departure', 'estimated_departure']:
    train_df[col] = pd.to_datetime(train_df[col]).dt.strftime('%a %b %d, %I:%M%p')

train_df

,train_num,heading,dest,origin,route,state,velocity,status,actual_arrival,scheduled_arrival,estimated_arrival,actual_departure,scheduled_departure,estimated_departure
0,5,SW,EMY,CHI,California Zephyr,Active,7.31974983215332,departed,"Mon Jun 03, 08:32AM","Mon Jun 03, 07:56AM",NaN,"Mon Jun 03, 09:18AM","Mon Jun 03, 08:46AM",NaN
1,6,W,CHI,EMY,California Zephyr,Active,15.0371799468994,enroute,NaN,"Tue Jun 04, 06:27PM","Tue Jun 04, 07:07PM",NaN,"Tue Jun 04, 06:59PM","Tue Jun 04, 07:29PM"
2,5,W,EMY,CHI,California Zephyr,Active,69.5314178466797,departed,"Tue Jun 04, 07:19AM","Tue Jun 04, 07:56AM",NaN,"Tue Jun 04, 08:46AM","Tue Jun 04, 08:46AM",NaN
3,6,N,CHI,EMY,California Zephyr,Active,79.0489883422852,enroute,NaN,"Wed Jun 05, 06:27PM","Wed Jun 05, 06:27PM",NaN,"Wed Jun 05, 06:59PM","Wed Jun 05, 06:59PM"
4,5,W,EMY,CHI,California Zephyr,Active,64.324333190918,enroute,NaN,"Wed Jun 05, 07:56AM","Wed Jun 05, 10:01AM",NaN,"Wed Jun 05, 08:46AM","Wed Jun 05, 10:35AM"
